# Tim Cools Eye data

### Create datafile

- subj
- trial
- value left
- value right
- maybe Qv left
- maybe Qv right
- prop viewing time left
- prop viewing time right
- first saccade (quantified as left/right) - which ROI was first "hit"; value will be "left", "right", "none"
- task-focus: 1 - if 5 trials before MW probe; 2 - if 5 trials before OT probe; 0 otherwise


### Hyp 1: first saccade to higher valued object?

Analysis:

- var1: if(first-saccade=left) left value, if(first-saccade=right), right value
- var2: the opposite (i.e., the value of the object that did NOT receive the first saccade)
- calculate diff=var1-var2
- mean(diff) per subject
- one-sample t-test with expect t>0

### Hyp 2: relationship viewing time vs. value

Analysis:

- var1: proportion viewing time to higher valued object (if left val>right val: left-viewing-time; if right val>left val: right-viewing-time)
- var2: abs(left value - right value)
- Correlation(var1,var2) per subject
- Fisher z-transform of the correlation coefficient for each subj
- one-sided t-test on all the z-transformed correlations (expect t>0)

### Hyp 3:  stimuli will receive more equal viewing time during periods of mind wandering than during periods of attention

- use var1 and var2 from Hyp2
- filter out, var1 and var2 where task-focus==MW or OT
- gets you OT_var1, MW_var1, OT_var2, MW_var2
- ANOVA: with factors OT vs. MW and var1 vs. var2
- hypothesis: IA is significant
- be prepared for lower power, because some people don't have MW trials




Hypotheses: value and viewing time
Hypotheses involving MW: filter by task-focus

In [3]:
import sys, os
sys.path.insert(0,"..") # this is not needed if you have installed pypillometry
import pypillometry as pp
print(pp.__package_path__) # check that correct version is being used
import pandas as pd
import numpy as np
import scipy
import pylab as plt
import glob

/Users/mmi041/Dropbox/work/projects/pupil/pypillometry/pypillometry


In [21]:
parent_path=os.path.expanduser("~/work/projects/mw/mw-learning/mwlearning_analysis/eyedata/")
data_path=os.path.join(parent_path, "asc/")
plot_dir=os.path.join(parent_path, "plots/")
subjects = ["%03i"%i for i in range(1,51)]
exclude_preproc=["018", "029", "032", "039", "044", "047", "049"]
subjs=list(set(subjects).difference(exclude_preproc))


In [20]:
def read_subject(subject, data_path=data_path):
    ## loading the raw samples from the asc file
    fname_samples=os.path.join(data_path, "{subj}_rlmw_samples.asc".format(subj=subject))
    df=pd.read_table(fname_samples, index_col=False,
                    names=["time", "left_x", "left_y", "left_p",
                            "right_x", "right_y", "right_p"])

    ## Eyelink tracker puts "   ." when no data is available for x/y coordinates
    left_x=df.left_x.values
    left_x[left_x=="   ."] = np.nan
    left_x = left_x.astype(float)

    left_y=df.left_y.values
    left_y[left_y=="   ."] = np.nan
    left_y = left_y.astype(float)

    right_x=df.right_x.values
    right_x[right_x=="   ."] = np.nan
    right_x = right_x.astype(float)

    right_y=df.right_y.values
    right_y[right_y=="   ."] = np.nan
    right_y = right_y.astype(float)

    ## Loading the events from the events file
    fname_events=os.path.join(data_path, "{subj}_rlmw_events.asc".format(subj=subject))
    # read the whole file into variable `events` (list with one entry per line)
    with open(fname_events) as f:
        events=f.readlines()

    # keep only lines starting with "MSG"
    events=[ev for ev in events if ev.startswith("MSG")]
    experiment_start_index=np.where(["experiment_start" in ev for ev in events])[0][0]
    events=events[experiment_start_index+1:]
    df_ev=pd.DataFrame([ev.split() for ev in events])
    df_ev=df_ev[[1,2]]
    df_ev.columns=["time", "event"]

    # Creating EyeData object that contains both X-Y coordinates
    # and pupil data
    d = pp.EyeData(time=df.time, name="{subj}_ceylin".format(subj=subject),
                screen_resolution=(1280,1024), physical_screen_size=(33.75,27),
                screen_eye_distance=60,
                left_x=left_x, left_y=left_y, left_pupil=df.left_p,
                right_x=right_x, right_y=right_y, right_pupil=df.right_p,
                event_onsets=df_ev.time, event_labels=df_ev.event,
                keep_orig=True)\
                .reset_time()
    d.set_experiment_info(screen_eye_distance=60, 
                        screen_resolution=(1280,1024), 
                        physical_screen_size=(30, 20))
    return d

In [33]:
# load pre-processed pupil data from Ceylin
pdata = pp.pd_read_pickle(os.path.join(parent_path, "pupildata_2024_03_19.pkl"))
pdata = {k:v for k,v in pdata.items() if k in subjs}

In [ ]:
edata={}
for subject in subjs:
    print("Processing subject %s"%subject)

    d = read_subject(subject)
    # remove the interpolated points from the pupil processing from saccade data
    for key in d.data.keys():
        d.data[key][pdata[subject].interpolated_mask==1]=np.nan
    edata[subject]=d

In [ ]:
## prop_viewing time left and right
#  - operationalize as proportion of time spent looking at left and right side of screen

In [129]:
def trial_stim_choice(key, trial=None):
    tup=key.split("_")
    ev=tup[0]; ctrial=tup[1]
    return (ev=="S" or ev=="C") and ctrial==str(trial)

d.test_selector(trial_stim_choice, trial=10)

def get_stim_choice_interval(d):
    """get interval between stimulus onset and choice"""
    ntrials=300
    ivs=[]
    for i in range(ntrials):
        iv = d.get_intervals(trial_stim_choice, padding=(0,1), trial=i+1)
        ivs.append( (iv[0][0], iv[1][0]) )
    return ivs

outdata = []

for subj in sorted(subjs):
    print("%s "%subj, end="")
    ivs=get_stim_choice_interval(d)
    screenmid = d.screen_xlim[1]/2

    propleft=[]
    propright=[]
    firstsacc=[]
    for start,end in ivs:
        startix=np.argmin(np.abs(d.tx-start))
        endix=np.argmin(np.abs(d.tx-end))

        ## proportion left and right
        nleft = np.sum(d.data["left_x"][startix:endix] < screenmid)
        nright = np.sum(d.data["left_x"][startix:endix] > screenmid)
        propleft.append(nleft/(nleft+nright)*100)
        propright.append(nright/(nleft+nright)*100)

        ## first saccade left or right
        # when x-coord of first sample comes closes to leftstim/rightstim 
        leftstim=screenmid-(screenmid/2)
        rightstim=screenmid+(screenmid/2)
        fsacc="none"
        ixleft = np.where(d.data["left_x"][startix:endix]<leftstim+200)
        if len(ixleft[0])>0:
            ixleft=ixleft[0][0]
        else:
            ixleft=np.nan
        ixright= np.where(d.data["left_x"][startix:endix]>rightstim-200)
        if len(ixright[0])>0:
            ixright=ixright[0][0]
        else:
            ixright=np.nan
        if not (np.isnan(ixleft) and np.isnan(ixright)):
            fsacc = ["left","right"][np.nanargmin([ixleft,ixright])]
        firstsacc.append(fsacc)
    
    out = {"subj":subj,"trial":np.arange(300)+1, 
           "propleft":propleft, "propright":propright,
           "firstsaccade":firstsacc}
    outdata.append(out)




001 002 003 004 005 006 007 008 009 010 011 012 013 014 015 016 017 019 020 021 022 023 024 025 026 027 028 030 031 033 034 035 036 037 038 040 041 042 043 045 046 048 050 

In [130]:
odata = pd.concat([pd.DataFrame(out) for out in outdata])

In [128]:
odata.to_csv(os.path.join(parent_path, "tim_prop_viewing_time.csv"), index=False)